In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import cv2

import os
from torchvision import transforms as T
import pandas as pd
from Utils.Dataloader import CropDataset
import matplotlib.pyplot as plt
from Utils.TextToImage import unique_char_mapping
from Utils.Filters import applyTransforms

from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

from SketchKeras.sketch_model import SketchKeras
import Utils.sketchkeras as SK 
import Utils.Predict as P 

PATCH_SIZE = 32

image = Image.open("image3.png").resize((2048, 2048))
sketch = Image.open("sketch3.png").resize((2048, 2048))
ascii_lines = Image.open("output3.png").resize((2048, 2048))
bw = Image.open("sit_toyosu.png").convert("L").resize((2048, 2048))
tone = Image.open("converted_sit_toyosu.png").resize((2048, 2048))

In [23]:
def extract_patches(image, n, m, filename=None):  
    patches = []
    image = np.array(image,float)
    height, width = image.shape[:2]
    images_height =0
    images_width = 0

    for y in range(0, height - n + 1, m):
        images_height += 1
        for x in range(0, width - n + 1, m):  
            images_width += 1          
            patch = image[y:y+n, x:x+n]
            patches.append(patch)
    images_width = images_width//images_height
    
    if filename:
        os.makedirs(filename, exist_ok=True)
        for i, patch in enumerate(patches):
            cv2.imwrite(f"{filename}/{filename}_{i}.png", patch)

    return np.array([patch for patch in patches]), images_width, images_height

In [24]:
# Assuming you have a list of image paths called 'image_paths'
# Load the images
images, _, _ = extract_patches(np.array(ascii_lines), PATCH_SIZE, PATCH_SIZE)
intra_cluster_variances = []

for image in images:
    kmeans = KMeans(n_clusters=8)  # You can adjust the number of clusters as needed
    kmeans.fit(image)

    # Get the cluster labels
    labels = kmeans.labels_

    # Find the irregularities
    irregularities = np.where(labels != labels[0])[0]

    # Print the irregularities
    intra_cluster_variance = kmeans.inertia_
    # print("Intra-cluster variance:", intra_cluster_variance)
    intra_cluster_variances.append(intra_cluster_variance)

/home/pablo/miniconda3/envs/ascii/lib/python3.10/site-packages/sklearn/base.py:1351: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/pablo/miniconda3/envs/ascii/lib/python3.10/site-packages/sklearn/base.py:1351: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/pablo/miniconda3/envs/ascii/lib/python3.10/site-packages/sklearn/base.py:1351: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/home/pablo/miniconda3/envs/ascii/lib/python3.10/site-packages/sklearn/base.py:1351: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit

In [25]:
intra_cluster_variances = np.array(intra_cluster_variances).mean()
print("Intra-cluster variance:", intra_cluster_variances)

Intra-cluster variance: 27567.382645058577
